### Front-Running Detector
Front-running is defined as trading a stock or another financial asset by a broker who has inside knowledge of a future transaction that is about to affect its price substantially. It is illegal and unethical.

Here's an example of front-running: Say a trader gets an order from abroker to buy 50,000 shares of Tesla. Such a huge purchase is bound to drive up the price of the stock immediately, at least in the short term. The trader sets aside the broker request for a minute and first buys some Tesla stock for their own porfolio. Then the broker's order is executed. The trader could then immediately sell the Tesla shares and collect a profit. The trader has make a profit bases on information that was not public knowledge.

Your task is to create a Front-Running Detector that will process option trade data from different exchanges and determine if front-running has occurred. Your solution should be able to handle data from multiple exchanges, with the expectation that additional exchanges will need to be supported in the future. Your implementation should follow good OOP design practices. Given a trade feed, output a list of all( broker_trade_id, electronic_trade_id) pairs. Trade pairs should be orderd by the electronic trade time.

A trade pair is considered front-running if all of the following conditions are met:
- a. One trade is of typer 'Broker' and the second trade is of type 'Electronic;.
- b. The Electronic trade occurs 1 minute or less before the Broker trade
- c. Both trades are for the same product.
- d. Both trades are of the same type (Call/Put).
- e. Both trades have the same side (Buy/Sell).
- f. Both trades have the same expiry date.
- g. Both trades have the same strike price.

Note. The incoming trades from CBOE do not have a side field, instead the quantity represents the side A positive qty represents a buy and negative represents a sell.

Example:

Trade1:(date = '2022-03-15',time=9:01:00,type=Broker, qty=-500,stike = 1500,expiry='2022-04-28',kind=P,exchange=CBOE,trade-id=737acm,product=ABC)
Trade2:(date = '2022-03-15',time=9:00:24,type=Electronic, qty=-200,stike = 1500,expiry='2022-04-28',kind=P,exchange=CBOE,trade-id=w6c229,product=ABC)
Trade3:(date = '2022-03-15',time=9:03:45,type=Electronic, qty=-100,stike = 1500,expiry='2022-04-28',kind=P,exchange=CBOE,trade-id=tssrin,product=ABC)[Fails condition(b)]
Trade4:(date = '2022-03-15',time=9:00:53,type=Electronic, qty=-500,stike = 1500,expiry='2022-04-28',kind=P,exchange=CBOE,trade-id=lk451a,product=XYZ)[Fails condition(c)]
Trade5:(date = '2022-03-15',time=9:00:05,type=Electronic, qty=-350,stike = 1500,expiry='2022-04-28',kind=C,exchange=CBOE,trade-id=9numpr,product=ABC)[Fails condition(d)]
Trade6:(date = '2022-03-15',time=9:00:35,type=Electronic, qty=20,stike = 1500,expiry='2022-04-28',kind=P,exchange=CBOE,trade-id=922v3g,product=ABC)[Fails condition(e)]
Trade7:(date = '2022-03-15',time=9:00:47,type=Electronic, qty=-150,stike = 1500,expiry='2022-04-21',kind=P,exchange=CBOE,trade-id=bg54nm,product=ABC)[Fails condition(f)]
Trade8:(date = '2022-03-15',time=9:01:00,type=Electronic, qty=-500,stike = 1550,expiry='2022-04-28',kind=P,exchange=CBOE,trade-id=6y7fhm,product=ABC)[Fails condition(g)]

OUTPUT:
[('737ACM,'W6C229)]




In [31]:
from typing import List, Tuple
import datetime
'''
RAW_TRADE_HEADER = ['trade_id','trade_date','time_of_trade',
'portfolio','exchange','product','product_type','expiry_dt',
'qty','strike_price','side']
'''
class Solution:
    # def __init__(self,raw_trade):
    #     self.raw_trade=raw_trade

    def process_raw_trade(self, raw_trade:List):
        Brockers = []
        Electronics = []
        for i in range(len(raw_trade)):
            trade = {}
            trade['id'] = raw_trade[i][0]
            trade['date'] = raw_trade[i][1]
            trade['time'] =raw_trade[i][2]
            trade['kind'] = raw_trade[i][3]
            trade['exchange'] = raw_trade[i][4]
            trade['product'] = raw_trade[i][5]
            trade['type'] = raw_trade[i][6]
            trade['expiry'] = raw_trade[i][7]
            trade['qty'] = raw_trade[i][8]
            trade['strike'] = raw_trade[i][9]
            trade['side'] = raw_trade[i][10]
            if trade['type'].lower() == 'broker':
                Brockers.append(trade)
            elif trade['type'].lower() == 'electronic':
                Electronics.append(trade)
        # print('Brockers:',Brockers)
        # print('Electronics:',Electronics)
        
        return [Brockers,Electronics]

    def run(self)-> List[Tuple[str,str]]:
        result = []
        a = self.process_raw_trade(raw_trade)
        for b in a[0]:
            for e in a[1]:
                dt_b = datetime.datetime.strptime(b['date']+ ' ' + b['time'], '%Y-%m-%d %H:%M:%S')
                dt_e = datetime.datetime.strptime(e['date']+ ' ' + e['time'], '%Y-%m-%d %H:%M:%S')
                
                # check the condition b : time_diff<= 1 min
                if (dt_b - dt_e).seconds >60:
                    continue
                # check the condition c : same product
                if b['product'] != e['product']:
                    continue
                # check the condition d : same type
                if b['kind'] != e['kind']:
                    continue
                # check the condition e : same side
                if b['side'] != e['side']:
                    continue
                # check the condition f : same expiry date
                if b['expiry'] != e['expiry']:
                    continue
                # check the condition g : same strike
                if b['strike'] != e['strike']:
                    continue
                result.append((b['id'],e['id']))
        return result


raw_trade = [[ '737acm','2022-03-15','09:01:00','P','CBOE','ABC' ,'Broker','2022-04-28', '500','1500','Sell'],
             [ 'w6c229','2022-03-15','09:00:30','P','CBOE','ABC' ,'Electronic','2022-04-28', '500','1500','Sell'],
             [ 'w6c229','2022-03-15','09:00:30','P','CBOE','ABC' ,'Electronic','2022-04-28', '500','1500','Buy']]
example = Solution()
example.process_raw_trade(raw_trade)
example.run()


[('737acm', 'w6c229')]

- date = '2022-03-15'  'trade_date'
- time=9:01:00  'time_of_trade'
- type=Broker  'product_type'
- qty=-500  'qty' ， 'side'
- stike = 1500  'strike_price'
- expiry='2022-04-28'  'expiry_dt'
- kind=P  'portfolio'
- exchange=CBOE  'exchange'
- trade-id=737acm  'trade_id'
- product=ABC  'product'
